### Bibliotecas

In [1]:
"""!pip install aeon
!pip install tsfresh
!pip install tslearn"""

'!pip install aeon\n!pip install tsfresh\n!pip install tslearn'

In [2]:
import pandas as pd
import numpy as np

from aeon.datasets import load_classification
from aeon.datasets.tsc_data_lists import univariate_equal_length
from aeon.classification.distance_based import KNeighborsTimeSeriesClassifier, ShapeDTW, ElasticEnsemble

from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import MinimalFCParameters

from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import PiecewiseAggregateApproximation, SymbolicAggregateApproximation

import pywt

from sklearn.metrics import accuracy_score

from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB

from scipy.fftpack import fft
from numba import jit
from tqdm import tqdm
import timeit
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")

### Time Series Classifier Dynamic Series Representation

In [ ]:
class TimeSeriesClassifier:
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.trained_models = {}
        """_summary_

        Returns:
            _type_: _description_
        """
    @jit
    def transform_data_math(X):
        n_sax_symbols = int(X.shape[1] / 4)
        n_paa_segments = int(X.shape[1] / 4)
    
        X_fft = np.abs(fft(X, axis=1))
    
        coeffs_cA, coeffs_cD = pywt.dwt(X, 'db1', axis=1)
        X_dwt = np.hstack((coeffs_cA, coeffs_cD))
    
        paa = PiecewiseAggregateApproximation(n_segments=n_paa_segments)
        X_paa_ = paa.inverse_transform(paa.fit_transform(X))
        X_paa = X_paa_.reshape(X_paa_.shape[0], -1)
        stats_PAA = np.hstack([np.mean(X_paa, axis=1).reshape(-1,1),
                               np.std(X_paa, axis=1).reshape(-1,1),
                               np.max(X_paa, axis=1).reshape(-1,1),
                               np.min(X_paa, axis=1).reshape(-1,1),
                               ])
    
        sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)
        X_sax_ = sax.inverse_transform(sax.fit_transform(X))
        X_sax = X_sax_.reshape(X_sax_.shape[0], -1)
        stats_SAX = np.hstack([np.mean(X_sax, axis=1).reshape(-1,1),
                               np.std(X_sax, axis=1).reshape(-1,1),
                               np.max(X_sax, axis=1).reshape(-1,1),
                               np.min(X_sax, axis=1).reshape(-1,1),
                               ])
    
        data_X = TimeSeriesScalerMeanVariance().fit_transform(X)
        data_X.resize(data_X.shape[0], data_X.shape[1])
        stats_X = np.hstack([np.mean(data_X, axis=1).reshape(-1,1),
                             np.std(data_X, axis=1).reshape(-1,1),
                             np.max(data_X, axis=1).reshape(-1,1),
                             np.min(data_X, axis=1).reshape(-1,1),
                             ])
    
        data_FFT = TimeSeriesScalerMeanVariance().fit_transform(X_fft)
        data_FFT.resize(data_FFT.shape[0], data_FFT.shape[1])
        stats_FFT = np.hstack([np.mean(data_FFT, axis=1).reshape(-1,1),
                               np.std(data_FFT, axis=1).reshape(-1,1),
                               np.max(data_FFT, axis=1).reshape(-1,1),
                               np.min(data_FFT, axis=1).reshape(-1,1),
                               ])
    
        data_DWT = TimeSeriesScalerMeanVariance().fit_transform(X_dwt)
        data_DWT.resize(data_DWT.shape[0], data_DWT.shape[1])
        stats_DWT = np.hstack([np.mean(data_DWT, axis=1).reshape(-1,1),
                               np.std(data_DWT, axis=1).reshape(-1,1),
                               np.max(data_DWT, axis=1).reshape(-1,1),
                               np.min(data_DWT, axis=1).reshape(-1,1),
                               ])
    
        return {
            "TS": np.hstack([data_X, stats_X]),
            "FFT": np.hstack([data_FFT, stats_FFT]),
            "DWT": np.hstack([data_DWT, stats_DWT]),
            "PAA": np.hstack([X_paa, stats_PAA]),
            "SAX": np.hstack([X_sax, stats_SAX])
        }
    
    @jit
    def select_model(option, random_state):
        if option == '1nn':
            return KNeighborsTimeSeriesClassifier(distance='euclidean', n_neighbors=1, n_jobs=-1)
        elif option == '3nn':
            return KNeighborsTimeSeriesClassifier(distance='dtw', n_neighbors=3, n_jobs=-1)
        elif option == 'svm':
            return SVC(C = 100, gamma=0.01, kernel='linear', probability=True)
        elif option == 'gbc':
            return GradientBoostingClassifier(n_estimators=5, random_state=random_state)
        elif option == 'nb':
            return GaussianNB()
        elif option == 'shape':
            return ShapeDTW(n_neighbors=1)
        elif option == 'ee':
            return ElasticEnsemble(n_jobs=-1,
                                   random_state=random_state,
                                   majority_vote=True)
        elif option == 'exrf':
            return ExtraTreesClassifier(n_estimators=200,
                                        criterion="entropy",
                                        class_weight="balanced",
                                        max_features="sqrt",
                                        n_jobs=-1,
                                        random_state=None)
        elif option == 'rd':
            return RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        else:
            return RandomForestClassifier(n_estimators=200,
                                          criterion="entropy",
                                          class_weight="balanced",
                                          max_features="sqrt",
                                          n_jobs=-1,
                                          random_state=None)
    
    @jit
    def train_with_meta_classifier(X_train, y_train, base_option='random_forest', meta_option='1nn', random_state=42):
        trained_models = {}  # Salvar modelos treinados para cada transformação
        X_train_transformed = transform_data_math(X_train)  # Transformar todo o conjunto de treino
        loo = LeaveOneOut()
        num_classes = len(np.unique(y_train))  # Número de classes
    
        # Treinar um modelo para cada transformação e salvar no dicionário
        for rep, X_trans in tqdm(X_train_transformed.items(), ascii=True, desc="Training Base Models"):
            model = select_model(base_option, random_state)
            scores = []
            for train_index, _ in loo.split(X_trans):
                model.fit(X_trans[train_index], y_train[train_index])
                score = model.score(X_trans[train_index], y_train[train_index])  # Score do modelo nos dados de treino
                scores.append(score)
            avg_score = np.mean(scores)
            trained_models[rep] = model  # Salvar o modelo treinado
    
        # Preparar dados para o meta-classificador
        meta_features = np.zeros((X_train.shape[0], num_classes))  # Inicializar um vetor para armazenar as somas de probabilidades para cada classe
        for i in range(X_train.shape[0]):
            for rep, model in trained_models.items():
                proba = model.predict_proba(X_train_transformed[rep][i].reshape(1, -1))
                meta_features[i] += proba.flatten()  # Adicione as probabilidades para cada classe
    
        # Treinar o meta-classificador
        meta_classifier = select_model(meta_option, random_state)
        meta_classifier.fit(meta_features, y_train)
    
        return trained_models, meta_classifier
    
    @jit
    def predict_with_meta_classifier(X_test, trained_base_models, trained_meta_classifier):
        predictions = []
        meta_features_test = []  # Inicialize uma lista para armazenar todos os meta-recursos dos dados de teste
    
        for i in tqdm(range(len(X_test)), ascii=True, desc="Testing Instances"):
            x_instance = X_test[i].reshape(1, -1)
            x_transformed = transform_data_math(x_instance)
    
            instance_features = np.zeros(trained_meta_classifier.n_features_in_)  # Inicialize um vetor para armazenar as características do meta-classificador
            for rep, model in trained_base_models.items():
                proba = model.predict_proba(x_transformed[rep][0].reshape(1, -1))
                instance_features += proba.flatten()  # Adicione as probabilidades para cada classe
    
            meta_feature = np.array(instance_features).reshape(1, -1)
            predictions.append(trained_meta_classifier.predict(meta_feature)[0])  # Adicionar a previsão à lista de previsões
    
            meta_features_test.append(meta_feature.flatten())  # Adicionar meta-recursos da instância atual à lista
    
        # Converter a lista de meta-recursos dos dados de teste em um array numpy
        meta_features_test = np.array(meta_features_test)
    
        # Salvar todos os meta-recursos dos dados de teste em um arquivo CSV
        np.savetxt("meta-features-test.csv", meta_features_test, delimiter=",")
    
        return predictions



### Dados

In [3]:
"""try:
    train_data = pd.read_parquet('D:\_MESTRADO\_Meta_Learning\MSC\CSV_Parquet\Car_TRAIN.parquet')
    test_data = pd.read_parquet('D:\_MESTRADO\_Meta_Learning\MSC\CSV_Parquet\Car_TRAIN.parquet')
except FileNotFoundError:
    print("Ensure the Parquet files are in the correct path.")
    raise


X_train = train_data.drop('target', axis=1).values
y_train = train_data['target'].values

X_test = test_data.drop('target', axis=1).values
y_test = test_data['target'].values"""


"""X_train, y_train = load_classification('CBF', split="TRAIN")
X_test, y_test = load_classification('CBF', split="test")

# Achatando os dados para 2D, pois alguns algoritmos esperam 2D
Xtrain = X_train.reshape(X_train.shape[0], -1)
Xtest = X_test.reshape(X_test.shape[0], -1)"""


'X_train, y_train = load_classification(\'CBF\', split="TRAIN")\nX_test, y_test = load_classification(\'CBF\', split="test")\n\n# Achatando os dados para 2D, pois alguns algoritmos esperam 2D\nXtrain = X_train.reshape(X_train.shape[0], -1)\nXtest = X_test.reshape(X_test.shape[0], -1)'

### Função de transformação dos dados.

In [4]:
@jit
def transform_data_math(X):
    n_sax_symbols = int(X.shape[1] / 4)
    n_paa_segments = int(X.shape[1] / 4)

    X_fft = np.abs(fft(X, axis=1))

    coeffs_cA, coeffs_cD = pywt.dwt(X, 'db1', axis=1)
    X_dwt = np.hstack((coeffs_cA, coeffs_cD))

    paa = PiecewiseAggregateApproximation(n_segments=n_paa_segments)
    X_paa_ = paa.inverse_transform(paa.fit_transform(X))
    X_paa = X_paa_.reshape(X_paa_.shape[0], -1)
    stats_PAA = np.hstack([np.mean(X_paa, axis=1).reshape(-1,1),
                           np.std(X_paa, axis=1).reshape(-1,1),
                           np.max(X_paa, axis=1).reshape(-1,1),
                           np.min(X_paa, axis=1).reshape(-1,1),
                           ])

    sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)
    X_sax_ = sax.inverse_transform(sax.fit_transform(X))
    X_sax = X_sax_.reshape(X_sax_.shape[0], -1)
    stats_SAX = np.hstack([np.mean(X_sax, axis=1).reshape(-1,1),
                           np.std(X_sax, axis=1).reshape(-1,1),
                           np.max(X_sax, axis=1).reshape(-1,1),
                           np.min(X_sax, axis=1).reshape(-1,1),
                           ])

    data_X = TimeSeriesScalerMeanVariance().fit_transform(X)
    data_X.resize(data_X.shape[0], data_X.shape[1])
    stats_X = np.hstack([np.mean(data_X, axis=1).reshape(-1,1),
                         np.std(data_X, axis=1).reshape(-1,1),
                         np.max(data_X, axis=1).reshape(-1,1),
                         np.min(data_X, axis=1).reshape(-1,1),
                         ])

    data_FFT = TimeSeriesScalerMeanVariance().fit_transform(X_fft)
    data_FFT.resize(data_FFT.shape[0], data_FFT.shape[1])
    stats_FFT = np.hstack([np.mean(data_FFT, axis=1).reshape(-1,1),
                           np.std(data_FFT, axis=1).reshape(-1,1),
                           np.max(data_FFT, axis=1).reshape(-1,1),
                           np.min(data_FFT, axis=1).reshape(-1,1),
                           ])

    data_DWT = TimeSeriesScalerMeanVariance().fit_transform(X_dwt)
    data_DWT.resize(data_DWT.shape[0], data_DWT.shape[1])
    stats_DWT = np.hstack([np.mean(data_DWT, axis=1).reshape(-1,1),
                           np.std(data_DWT, axis=1).reshape(-1,1),
                           np.max(data_DWT, axis=1).reshape(-1,1),
                           np.min(data_DWT, axis=1).reshape(-1,1),
                           ])

    return {
        "TS": np.hstack([data_X, stats_X]),
        "FFT": np.hstack([data_FFT, stats_FFT]),
        "DWT": np.hstack([data_DWT, stats_DWT]),
        "PAA": np.hstack([X_paa, stats_PAA]),
        "SAX": np.hstack([X_sax, stats_SAX])
    }


### Seleção do modelo extrator e modelo classificador

In [5]:
@jit
def select_model(option, random_state):
    if option == '1nn':
        return KNeighborsTimeSeriesClassifier(distance='euclidean', n_neighbors=1, n_jobs=-1)
    elif option == '3nn':
        return KNeighborsTimeSeriesClassifier(distance='dtw', n_neighbors=3, n_jobs=-1)
    elif option == 'svm':
        return SVC(C = 100, gamma=0.01, kernel='linear', probability=True)
    elif option == 'gbc':
        return GradientBoostingClassifier(n_estimators=5, random_state=random_state)
    elif option == 'nb':
        return GaussianNB()
    elif option == 'shape':
        return ShapeDTW(n_neighbors=1)
    elif option == 'ee':
        return ElasticEnsemble(n_jobs=-1,
                               random_state=random_state,
                               majority_vote=True)
    elif option == 'exrf':
        return ExtraTreesClassifier(n_estimators=200,
                                    criterion="entropy",
                                    class_weight="balanced",
                                    max_features="sqrt",
                                    n_jobs=-1,
                                    random_state=None)
    elif option == 'rd':
        return RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
    else:
        return RandomForestClassifier(n_estimators=200,
                                      criterion="entropy",
                                      class_weight="balanced",
                                      max_features="sqrt",
                                      n_jobs=-1,
                                      random_state=None)
        #return RandomForestClassifier(n_estimators=100,random_state=random_state)

### Treino do modelos extrator e classificador

In [6]:
"""@jit
def train_with_meta_classifier(X_train, y_train, base_option='random_forest', meta_option='1nn', random_state=42):
    trained_models = {}  # Salvar modelos treinados para cada transformação
    best_model_indices = []
    best_probabilities = []
    X_train_transformed = transform_data_math(X_train)  # Transformar todo o conjunto de treino
    loo = LeaveOneOut()

    # Treinar um modelo para cada transformação e salvar no dicionário
    for rep, X_trans in tqdm(X_train_transformed.items(), ascii=True, desc="Training Base Models"):
        model = select_model(base_option, random_state)
        scores = []
        for train_index, _ in loo.split(X_trans):
            model.fit(X_trans[train_index], y_train[train_index])
            score = model.score(X_trans[train_index], y_train[train_index])  # Score do modelo nos dados de treino
            scores.append(score)
        avg_score = np.mean(scores)
        trained_models[rep] = (model, avg_score)  # Salvar o modelo treinado e a média dos scores

    # Preparar dados para o meta-classificador
    meta_features = []
    for i in range(X_train.shape[0]):
        instance_features = []
        for rep, (model, _) in trained_models.items():
            proba = model.predict_proba(X_train_transformed[rep][i].reshape(1, -1))
            instance_features.extend(proba.flatten())  # Estender a lista com todas as probabilidades

        meta_features.append(instance_features)

        best_model_index = None
        best_accuracy = -1

        # Iterar sobre os modelos treinados e suas respectivas probabilidades
        for rep, (model, _) in trained_models.items():
            proba = model.predict_proba(X_train_transformed[rep][i].reshape(1, -1))
            accuracy = np.max(proba)  # Calcula a precisão usando a probabilidade máxima

            # Se a precisão desse modelo for a melhor até agora, armazene o índice do modelo e as probabilidades
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model_index = rep
                best_proba = proba.flatten()

        # Adicione o índice do modelo que obteve a melhor precisão e as probabilidades correspondentes às listas
        best_model_indices.append(best_model_index)
        best_probabilities.append(best_proba)

    # Converta as listas em arrays NumPy
    best_model_indices = np.array(best_model_indices)
    best_probabilities = np.array(best_probabilities)

    # Crie uma lista para armazenar as meta_features usando apenas as melhores probabilidades
    best_meta_features = []

    # Iterar sobre as melhores probabilidades para cada instância e adicionar apenas essas probabilidades às meta_features
    for i in range(len(best_probabilities)):
        best_meta_features.append(best_probabilities[i])

    # Converta a lista em um array NumPy
    best_meta_features = np.array(best_meta_features)

    meta_features = np.array(meta_features)
    np.savetxt("meta-features-train.csv", meta_features, delimiter=",")

    # Treinar o meta-classificador
    meta_classifier = select_model(meta_option, random_state)
    meta_classifier.fit(meta_features, y_train)

    return trained_models, meta_classifier
"""

'@jit\ndef train_with_meta_classifier(X_train, y_train, base_option=\'random_forest\', meta_option=\'1nn\', random_state=42):\n    trained_models = {}  # Salvar modelos treinados para cada transformação\n    best_model_indices = []\n    best_probabilities = []\n    X_train_transformed = transform_data_math(X_train)  # Transformar todo o conjunto de treino\n    loo = LeaveOneOut()\n\n    # Treinar um modelo para cada transformação e salvar no dicionário\n    for rep, X_trans in tqdm(X_train_transformed.items(), ascii=True, desc="Training Base Models"):\n        model = select_model(base_option, random_state)\n        scores = []\n        for train_index, _ in loo.split(X_trans):\n            model.fit(X_trans[train_index], y_train[train_index])\n            score = model.score(X_trans[train_index], y_train[train_index])  # Score do modelo nos dados de treino\n            scores.append(score)\n        avg_score = np.mean(scores)\n        trained_models[rep] = (model, avg_score)  # Salvar

In [7]:
@jit
def train_with_meta_classifier(X_train, y_train, base_option='random_forest', meta_option='1nn', random_state=42):
    trained_models = {}  # Salvar modelos treinados para cada transformação
    X_train_transformed = transform_data_math(X_train)  # Transformar todo o conjunto de treino
    loo = LeaveOneOut()
    num_classes = len(np.unique(y_train))  # Número de classes

    # Treinar um modelo para cada transformação e salvar no dicionário
    for rep, X_trans in tqdm(X_train_transformed.items(), ascii=True, desc="Training Base Models"):
        model = select_model(base_option, random_state)
        scores = []
        for train_index, _ in loo.split(X_trans):
            model.fit(X_trans[train_index], y_train[train_index])
            score = model.score(X_trans[train_index], y_train[train_index])  # Score do modelo nos dados de treino
            scores.append(score)
        avg_score = np.mean(scores)
        trained_models[rep] = model  # Salvar o modelo treinado

    # Preparar dados para o meta-classificador
    meta_features = np.zeros((X_train.shape[0], num_classes))  # Inicializar um vetor para armazenar as somas de probabilidades para cada classe
    for i in range(X_train.shape[0]):
        for rep, model in trained_models.items():
            proba = model.predict_proba(X_train_transformed[rep][i].reshape(1, -1))
            meta_features[i] += proba.flatten()  # Adicione as probabilidades para cada classe

    # Treinar o meta-classificador
    meta_classifier = select_model(meta_option, random_state)
    meta_classifier.fit(meta_features, y_train)

    return trained_models, meta_classifier


### Predicao do meta-classificador

In [8]:
@jit
def predict_with_meta_classifier(X_test, trained_base_models, trained_meta_classifier):
    predictions = []
    meta_features_test = []  # Inicialize uma lista para armazenar todos os meta-recursos dos dados de teste

    for i in tqdm(range(len(X_test)), ascii=True, desc="Testing Instances"):
        x_instance = X_test[i].reshape(1, -1)
        x_transformed = transform_data_math(x_instance)

        instance_features = np.zeros(trained_meta_classifier.n_features_in_)  # Inicialize um vetor para armazenar as características do meta-classificador
        for rep, model in trained_base_models.items():
            proba = model.predict_proba(x_transformed[rep][0].reshape(1, -1))
            instance_features += proba.flatten()  # Adicione as probabilidades para cada classe

        meta_feature = np.array(instance_features).reshape(1, -1)
        predictions.append(trained_meta_classifier.predict(meta_feature)[0])  # Adicionar a previsão à lista de previsões

        meta_features_test.append(meta_feature.flatten())  # Adicionar meta-recursos da instância atual à lista

    # Converter a lista de meta-recursos dos dados de teste em um array numpy
    meta_features_test = np.array(meta_features_test)

    # Salvar todos os meta-recursos dos dados de teste em um arquivo CSV
    np.savetxt("meta-features-test.csv", meta_features_test, delimiter=",")

    return predictions


### Testando um único modelo - Random Forest como extrator e SVM como meta-classificador

In [9]:
dataset_full_list = ['Earthquakes','CricketX','Haptics','EOGVerticalSignal','MixedShapesSmallTrain','Meat','SemgHandGenderCh2','ToeSegmentation2','NonInvasiveFetalECGThorax1','FreezerSmallTrain','OSULeaf','BirdChicken','HandOutlines','BeetleFly','ACSF1','DistalPhalanxOutlineAgeGroup','FreezerRegularTrain', 'NonInvasiveFetalECGThorax2']

In [10]:
dataset_list = ['Adiac', 'Beef', 'Car', 'CBF', 'Coffee', 'DiatomSizeReduction', 'ECG200', 'ECGFiveDays', 'FaceFour','GunPoint', 'Lightning2', 'Lightning7', 'MoteStrain', 'OliveOil','MedicalImages', 'Trace', 'TwoPatterns']
#finalizado 'Adiac', 'Beef', 'Car', 'CBF', 'Coffee', 'DiatomSizeReduction', 'ECG200', 'ECGFiveDays', 'FaceFour','GunPoint', 'Lightning2', 'Lightning7', 'MoteStrain', 'OliveOil','MedicalImages', 'Trace'
# Para cada conjunto de dados na lista
for dataset_name in dataset_full_list:
    # Carregue os dados de treinamento e teste
    X_train, y_train = load_classification(dataset_name, split="TRAIN")
    X_test, y_test = load_classification(dataset_name, split="test")

    # Achatando os dados para 2D, pois alguns algoritmos esperam 2D
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)

    dataset_accuracies = []
    trained_base_models, meta_classifier = train_with_meta_classifier(X_train_flat, y_train, base_option='exrf', meta_option='rd')
    predictions_test_meta = predict_with_meta_classifier(X_test_flat, trained_base_models, meta_classifier)
    test_accuracy_meta = np.mean(predictions_test_meta == y_test)
    dataset_accuracies.append(test_accuracy_meta)

    print(f"Acurácia {dataset_name}: {test_accuracy_meta}")

#np.savetxt("Results_MSLOO_.csv", dataset_accuracies, delimiter=",")

Testing Instances: 100%|##########| 139/139 [00:22<00:00,  6.08it/s]


Acurácia Earthquakes: 0.7482014388489209


### Gráfico das diferenças de dados

In [ ]:
import matplotlib.pyplot as plt

y1 = y_hat  # depois da transformação
y2 = y_test

z1 = y_hat_ #antes da transformação
z2 = y_test

#suavizar os dados do gráfico
window_size = 15
y1_smoothed = pd.Series(y1).rolling(window=window_size).mean()
y2_smoothed = pd.Series(y2).rolling(window=window_size).mean()
z1_smoothed = pd.Series(z1).rolling(window=window_size).mean()
z2_smoothed = pd.Series(z2).rolling(window=window_size).mean()

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5), layout='constrained')

# Conjunto de validação do classificador
axs[0].set_title('Antes da transformação')
axs[0].plot(z1_smoothed, label='Treino')
axs[0].plot(z2_smoothed, label='Teste')
axs[0].set_xlabel('Tempo (s)')
axs[0].set_ylabel('Treino')
axs[0].grid(True)

# Conjunto de validação do meta-classificador
axs[1].set_title('Depois da transformação')
axs[1].plot(y1_smoothed, label='Treino')
axs[1].plot(y2_smoothed, label='Teste')
axs[1].set_xlabel('Tempo (s)')
axs[1].set_ylabel('Treino')
axs[1].grid(True)

plt.legend()
plt.show()



In [ ]:
w1 = y_hat  # meta-classificador
w2 = y_hat_ #classificação

# Suavizar os dados do gráfico
window_size = 15
w1_smoothed = pd.Series(w1).rolling(window=window_size).mean()
w2_smoothed = pd.Series(w2).rolling(window=window_size).mean()

# Plotar os dados
plt.figure(figsize=(10, 6))
plt.plot(w1_smoothed, label='w1 (Classificação usando meta-caracteristicas)')
plt.plot(w2_smoothed, label='w2 (classificação utilizando dados brutos)')
plt.xlabel('Tempo (s)')
plt.ylabel('Valores suavizados')
plt.title('Comparação entre os resultados de um SVM')
plt.legend()
plt.grid(True)
plt.show()


### Treino em loop de todas as opções de classificadores disponiveis no Select Model.

In [ ]:
algos = ['1nn', '3nn', 'svm', 'nb', 'gbc', 'ee', 'shape', 'rf', 'rd']
for algo in algos:

    print(f'Meta-classificador com modelo extrator {algo.upper()}')

    # Training
    try:
        trained_base_models, meta_classifier = train_with_meta_classifier(X_train, y_train, base_option='svm', meta_option=algo)
        # Testing
        predictions_test_meta = predict_with_meta_classifier(X_test, trained_base_models, meta_classifier)
        test_accuracy_meta = np.mean(predictions_test_meta == y_test)

        print(f'Acurácia do teste usando o meta-classificador com modelo extrator {algo}: {test_accuracy_meta}')
    except Exception as e:
        print(f"Ocorreu um erro no teste com o {algo}: {e}")
    print("-------------------------------")